In [1]:
import os
import mlflow
import mlflow.sklearn
import joblib
import pandas as pd
from pymongo import MongoClient
from datetime import datetime, timedelta

# ---------------------------------------------------------
# 1. FORCE AUTHENTICATION (Bypasses Browser/OAuth)
# ---------------------------------------------------------
# These are pulled from your GitHub Secrets via the YAML env block
os.environ["MLFLOW_TRACKING_USERNAME"] = os.environ.get("DAGSHUB_USERNAME", "Alihasnain388")
os.environ["MLFLOW_TRACKING_PASSWORD"] = os.environ.get("DAGSHUB_TOKEN")

# Hard-set the tracking URI
mlflow.set_tracking_uri("https://dagshub.com/Alihasnain388/AQI_Model.mlflow")

# Initialize the MLflow Client
client = mlflow.tracking.MlflowClient()

# ---------------------------------------------------------
# 2. LOAD LATEST REGISTERED MODEL & SCALER
# ---------------------------------------------------------
MODEL_NAME = "Karachi_AQI_Model"

print(f"🔍 Searching for the latest version of {MODEL_NAME}...")

try:
    # Get the latest version from the Model Registry
    versions = client.get_latest_versions(MODEL_NAME, stages=["None"])
    if not versions:
        raise Exception(f"No versions found for model {MODEL_NAME}")
    
    latest_version = versions[0].version
    run_id = versions[0].run_id
    
    print(f"✅ Found Model Version: {latest_version}")
    print(f"✅ Found Run ID: {run_id}")

    # Load the Model
    model_uri = f"models:/{MODEL_NAME}/{latest_version}"
    model = mlflow.sklearn.load_model(model_uri)

    # Download and Load the Scaler
    # We use the same run_id that the model was saved in
    scaler_path = mlflow.artifacts.download_artifacts(
        run_id=run_id, 
        artifact_path="model/scaler.pkl"
    )
    scaler = joblib.load(scaler_path)
    
    print("✅ Model and Scaler loaded successfully from DagsHub Registry.")

except Exception as e:
    print(f"❌ Error loading model/artifacts: {e}")
    exit(1)

# ---------------------------------------------------------
# 3. FETCH LATEST DATA FROM MONGODB
# ---------------------------------------------------------
MONGO_URI = os.getenv("MONGO_URI")
if not MONGO_URI:
    raise ValueError("❌ MONGO_URI environment variable is missing!")

mongo_client = MongoClient(MONGO_URI)
db = mongo_client["AQIPredictionSystem"]
collection = db["karachi_features"]

latest_data = collection.find_one(sort=[("timestamp", -1)])

if latest_data is None:
    raise ValueError("❌ No feature data found in MongoDB")

# ---------------------------------------------------------
# 4. PREPARE INPUT & PREDICT
# ---------------------------------------------------------
feature_columns = [
    "hour",
    "day_of_week",
    "aqi_lag_1h",
    "aqi_lag_24h",
    "aqi_change_rate",
    "Wind_Speed_kmh",
    "PM2.5_ugm3"
]

input_df = pd.DataFrame([latest_data])[feature_columns]

# Scale the input
scaled_input = scaler.transform(input_df)

# Predict (MultiOutputRegressor returns 72 values)
predictions = model.predict(scaled_input)[0]

# ---------------------------------------------------------
# 5. GENERATE 72-HOUR FORECAST
# ---------------------------------------------------------
start_time = datetime.now().replace(minute=0, second=0, microsecond=0) + timedelta(hours=1)

forecast_df = pd.DataFrame({
    "Time": [start_time + timedelta(hours=i) for i in range(72)],
    "Predicted_AQI": predictions
})

print("\n📈 Next 72 Hours AQI Forecast:")
print(forecast_df.to_string(index=False))

# ---------------------------------------------------------
# 6. (Optional) SAVE TO MONGODB OR CSV
# ---------------------------------------------------------
# forecast_dict = forecast_df.to_dict(orient="records")
# db["forecasts"].insert_many(forecast_dict)
# print("✅ Forecast saved to MongoDB")








TypeError: str expected, not NoneType